<a href="https://colab.research.google.com/github/daegeunbae/daegeunbae/blob/main/%EC%93%B0%EB%A0%88%EA%B8%B0%EB%B6%84%EB%A5%98%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%981_230112.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch
!pip3 install torchvision

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!ls
!ls "/gdrive/My Drive/Colab Notebooks/"

In [ ]:
import zipfile as zf
files = zf.ZipFile("/gdrive/My Drive/Colab Notebooks/dataset.zip",'r')
files.extractall()
files.close()

In [ ]:
!ls

In [ ]:
import os
os.listdir(os.path.join(os.getcwd(), "dataset"))

In [ ]:
from pathlib import Path
path = Path(os.getcwd())/"dataset"
path

In [ ]:
# train, valid, test 구조로 분할하는 함수
def split_indices(folder,seed1,seed2):    
    n = len(os.listdir(folder))
    full_set = list(range(1,n+1))

    # train 인덱싱
    random.seed(seed1)
    train = random.sample(list(range(1,n+1)), int(.5*n)) # 50%를 train 폴더로 이동
    remain = list(set(full_set)-set(train))

    # valid 인덱싱
    random.seed(seed2)
    valid = random.sample(remain, int(.5*len(remain))) # 남은 것들 중 50%(전체 25%)를 valid 폴더로 이동
    test = list(set(remain)-set(valid))
    
    return(train,valid,test)

# 인덱싱에 따라 파일 이름을 구하기
def get_names(waste_type, indices):
    file_names = [waste_type+str(i)+".jpg" for i in indices]
    return(file_names)

# 파일의 위치를 옮기는 함수
def move_files(source_files, destination_folder):
    for file in source_files:
        shutil.move(file, destination_folder)


In [ ]:

import random
import shutil

# 데이터 종류 / 쓰레기 타입 설정
subsets = ['train','valid']
waste_types = ['cardboard','glass','metal','paper','plastic','trash']

# data/데이터 종류/쓰레기 타입 디렉토리가 존재하지 않으면 폴더를 생성한다.
for subset in subsets:
    for waste_type in waste_types:
        folder = os.path.join('data', subset, waste_type)
        if not os.path.exists(folder):
            os.makedirs(folder)

# test 폴더는 쓰레기 타입 디렉토리가 포함되지 않으므로 따로 작성
if not os.path.exists(os.path.join('data','test')):
    os.makedirs(os.path.join('data','test'))
            
# 각 쓰레기 타입별로 
for waste_type in waste_types:
    source_folder = os.path.join('dataset', waste_type)
    train_ind, valid_ind, test_ind = split_indices(source_folder,1,1)
    
    # 파일들을 train 폴더로 옮김
    train_names = get_names(waste_type,train_ind)
    train_source_files = [os.path.join(source_folder,name) for name in train_names]
    move_files(train_source_files, "data/train/" + waste_type)
    
    # 파일들을 valid 폴더로 옮김
    valid_names = get_names(waste_type,valid_ind)
    valid_source_files = [os.path.join(source_folder,name) for name in valid_names]
    move_files(valid_source_files, "data/valid/" + waste_type)
    
    # 파일들을 test폴더로 옮김
    test_names = get_names(waste_type,test_ind)
    test_source_files = [os.path.join(source_folder,name) for name in test_names]
    move_files(test_source_files,"data/test")    # test 폴더는 쓰레기 타입 미포함

In [ ]:

# 변경된 디렉토리 구조 확인
!ls
!ls data

In [ ]:

!pip install fastai==1.0.42

In [ ]:
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from fastai import *

from fastai.vision import *


path = Path(os.getcwd())/"data"
tfms = get_transforms(do_flip=True, flip_vert=True)
data = ImageDataBunch.from_folder(path, test="test", ds_tfms=tfms, bs=16)
data
     

In [ ]:
data.classes

In [ ]:
data.show_batch(rows=4, figsize=(8,8))

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)